# Naive galaxy pair-counting with `Astropy`

In [1]:
%matplotlib inline

In [2]:
from astropy.coordinates import SkyCoord, Distance

## Computing the distance between points on the sky

`Astropy` has a built-in function for computing physical 3d distances in ${\rm Mpc}$ units from a pair of points with coordinates $\{{\rm ra_1, dec_1, z_1}\}$ and $\{{\rm ra_2, dec_2, z_2}\}.$ All you need to do is build a `SkyCoord` object with each point, and then you can compute the 3-d distance like this. 

In [3]:
#  Pick some random {ra, dec, z} values, giving them astropy.units
from astropy import units as u

ra1, dec1, z1 = 4*u.deg, 2.5*u.deg, 0.02
ra2, dec2, z2 = 5*u.deg, 1.5*u.deg, 0.025

In [4]:
#  Convert these {ra, dec, z} values to `SkyCoord` objects

sc1 = SkyCoord(ra1, dec1, Distance(z=z1))
sc2 = SkyCoord(ra2, dec2, Distance(z=z2))

In [5]:
#  Compute the distance using the built-in `separation_3d` method

sc1.separation_3d(sc2)

<Distance 22.510262241492832 Mpc>

The same syntax works if `sc2` stores an array of coordinates. 

In [6]:
npts2 = int(1e3)
sample2_ra = np.random.uniform(0.5, 2, npts2)*u.deg
sample2_dec = np.random.uniform(0.5, 2, npts2)*u.deg
sample2_z = Distance(z=np.random.uniform(0.02, 0.03, npts2))
sc2 = SkyCoord(sample2_ra, sample2_dec, Distance(z=sample2_z))

sc1.separation_3d(sc2)

<Distance [ 1373522.80542806, 1433923.04406852, 1595900.60854896,
            1703036.77859354, 1580464.52166329, 1526568.49490761,
            1261571.65278387, 1449030.31765067, 1205214.88341364,
            1393544.24906942, 1716384.40349167, 1605538.71638641,
            1292501.52761311, 1324213.26917834, 1319397.43056805,
            1392474.77254763, 1155288.58801122, 1244693.84677558,
            1354297.18073009, 1500869.34403424, 1254046.37287724,
            1334413.5991349 , 1570554.6307774 , 1190997.2965327 ,
            1295699.15226323, 1208940.88683912, 1558575.09168355,
            1376222.66532055, 1489548.30512764, 1363625.03040567,
            1671450.82461529, 1547938.79461526, 1159268.72536507,
            1729790.43589022, 1307037.12681376, 1424945.34411586,
            1184151.81428091, 1668705.35570347, 1300062.57523757,
            1330640.92959358, 1728382.27226271, 1515208.04780923,
            1275533.72728265, 1579623.54016806, 1675519.2291767 ,
          

But if `s1` also contains an array of coordinates, have to run a `for` loop over `sc1` since we can only do the array calculation one value of `sc1` at a time. 

In [7]:
npts1 = 10

sample1_ra = np.random.uniform(0.5, 2, npts1)*u.deg
sample1_dec = np.random.uniform(0.5, 2, npts1)*u.deg
sample1_z = Distance(z=np.random.uniform(0.02, 0.03, npts1))
sample1_coords = SkyCoord(sample1_ra, sample1_dec, Distance(z=sample1_z))

distances_3d = np.zeros(npts1*npts2)
index_counter = 0
for i, sc1 in enumerate(sample1_coords):
    s2_distances_to_i = sc1.separation_3d(sc2)
    distances_3d[index_counter:index_counter+npts2] = s2_distances_to_i
    index_counter = index_counter + npts2

distances_3d = distances_3d*u.Mpc
distances_3d

<Quantity [ 365507.69518596, 305374.1937853 , 143179.40432647,...,
            352631.47733623, 211688.77227191,  76705.64159581] Mpc>

So to pre-compute all pair distances, we can just feed our bcg positions as `sample1_coords`, and our cross-correlation sample will be `sample2_coords`. For us, we won't actually know the redshifts of the sample-2 galaxies. But we can still compute their projected distances by just artificially using the BCG redshift for the the sample-2 galaxies. 


In [8]:
proj_distances = np.zeros(npts1*npts2)
index_counter = 0
for i, sc1 in enumerate(sample1_coords):
    sc2 = SkyCoord(sample2_ra, sample2_dec, sc1.distance)
    s2_distances_to_i = sc1.separation_3d(sc2)
    proj_distances[index_counter:index_counter+npts2] = s2_distances_to_i
    index_counter = index_counter + npts2

proj_distances = proj_distances*u.Mpc
proj_distances

<Quantity [ 22931.13746814, 24856.15974699, 13871.94925262,...,
            30865.81238936, 17135.1731384 , 18828.23604712] Mpc>

# Timing tests

The calculation scales roughly linearly with the number of points in `sample1_coords`, but the total runtime depends sensitively on the number of `sample2_points`. Let's do a few timing tests. 

In [9]:
def fake_sc_data(num_bcg, num_crossgals):

    sample1_ra = np.random.uniform(0.5, 25, num_bcg)*u.deg
    sample1_dec = np.random.uniform(0.5, 25, num_bcg)*u.deg
    sample1_z = Distance(z=np.random.uniform(0.02, 0.1, num_bcg))
    sample1_coords = SkyCoord(sample1_ra, sample1_dec, Distance(z=sample1_z))

    sample2_ra = np.random.uniform(0.5, 25, num_crossgals)*u.deg
    sample2_dec = np.random.uniform(0.5, 25, num_crossgals)*u.deg

    return sample1_coords, sample2_ra, sample2_dec

def compute_projected_distances(sample1_coords, sample2_ra, sample2_dec):
    npts1 = len(sample1_coords)
    npts2 = len(sample2_ra)
    proj_distances = np.zeros(npts1*npts2)

    index_counter = 0
    for i, sc1 in enumerate(sample1_coords):
        sc2 = SkyCoord(sample2_ra, sample2_dec, sc1.distance)
        s2_distances_to_i = sc1.separation_3d(sc2)
        proj_distances[index_counter:index_counter+npts2] = s2_distances_to_i
        index_counter = index_counter + npts2

    proj_distances = proj_distances*u.Mpc
    return proj_distances


In [10]:
from time import time
def time_me(npts1, npts2):
    start = time()
    sc1, sample2_ra, sample2_dec = fake_sc_data(npts1, npts2)
    __ = compute_projected_distances(sc1, sample2_ra, sample2_dec)
    for i in range(len(sc1)):
        result = sc1[i].separation_3d(sc2)
    end = time()
    runtime = end-start
    msg = "For (npts1, npts2) = ({0}, {1}), runtime = {2:.2f} seconds"
    return msg.format(npts1, npts2, runtime)


In [11]:
num_bcg, num_crossgals = 10, int(1e3)
time_me(num_bcg, num_crossgals)

'For (npts1, npts2) = (10, 1000), runtime = 1.86 seconds'

In [12]:
print(time_me(10, int(1e2)))
print(time_me(10, int(1e3)))
print(time_me(10, int(1e4)))
print(time_me(10, int(1e5)))

For (npts1, npts2) = (10, 100), runtime = 1.72 seconds
For (npts1, npts2) = (10, 1000), runtime = 1.77 seconds
For (npts1, npts2) = (10, 10000), runtime = 1.76 seconds
For (npts1, npts2) = (10, 100000), runtime = 2.20 seconds


In [13]:
print(time_me(10, int(1e3)))
print(time_me(int(20), int(1e3)))
print(time_me(int(40), int(1e3)))
print(time_me(int(80), int(1e3)))

For (npts1, npts2) = (10, 1000), runtime = 1.77 seconds
For (npts1, npts2) = (20, 1000), runtime = 3.53 seconds
For (npts1, npts2) = (40, 1000), runtime = 14.23 seconds
For (npts1, npts2) = (80, 1000), runtime = 16.10 seconds
